# Transforming

Translate, correct spelling and grammer errors, convert format (to JSON, HTML), etc.

## Setup

In [1]:
!pip install openai
!pip install python-dotenv

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

# load ENVs from a local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
model = "gpt-3.5-turbo"

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0, # the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Translation

### Simple translation

In [4]:
prompt = f"""
Translate the following English text to German:
```Hi, I'd like to order a blender```
"""

response = get_completion(prompt)
print(response)

Hallo, ich möchte einen Mixer bestellen.


### Detect language

In [5]:
prompt = f"""
Tell me what language this is:
```Hallo, ich möchte einen Mixer bestellen.```
"""

response = get_completion(prompt)
print(response)

This is German.


### Translate into multiple languages

In [6]:
prompt = f"""
Translate the following English text to German, French, Chinese:
```Hi, I'd like to order a blender```
"""

response = get_completion(prompt)
print(response)

German: Hallo, ich würde gerne einen Mixer bestellen.
French: Bonjour, je voudrais commander un mixeur.
Chinese: 你好，我想订购一个搅拌机。


### Translate into formal and informal forms

In [7]:
prompt = f"""
Translate the following English text to Japanese in both the formal and informal forms:
```Hi, would you like to order a pillow?```
"""

response = get_completion(prompt)
print(response)

Formal: こんにちは、枕を注文されますか？
Informal: こんにちは、枕を注文する？


In [8]:
prompt = f"""
Translate the following English text to Chinese in both the formal and informal forms:
```Hi, would you like to order a pillow?```
"""

response = get_completion(prompt)
print(response)

Formal: 您好，您想订购枕头吗？
Informal: 嗨，你想订购枕头吗？


### Universal translator

In [9]:
user_messages = [
    "My screen is flashing",
    "Мой экран мигает",
    "Mein Bildschirm blinkt",
    "画面が点滅しています",
    "Mon écran clignote"
]

In [11]:
for issue in user_messages:
    prompt = f"Tell me what language this is in one word: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")
    
    prompt = f"""
        Translate the following text to English and Korean: 
        ```{issue}```
    """
    
    response = get_completion(prompt)
    print(response, "\n")

Original message (English): My screen is flashing
English: My screen is flashing
Korean: 내 화면이 깜빡입니다. 

Original message (Russian.): Мой экран мигает
English: My screen is flickering.
Korean: 내 화면이 깜빡입니다. 

Original message (German.): Mein Bildschirm blinkt
English: My screen is blinking
Korean: 내 화면이 깜빡입니다. 

Original message (Japanese.): 画面が点滅しています
English: The screen is flashing.

Korean: 화면이 깜빡입니다. 

Original message (French.): Mon écran clignote
English: My screen is flickering
Korean: 내 화면이 깜빡입니다. 



## Tone transformation

In [12]:
prompt = f"""
    Transform the following from slang to a business letter:
    "Dude, this is Joe. Check out this spec on this standing lamp."
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention a standing lamp that I believe may be of interest to you. Please find attached the specifications for your review.

Thank you for your time and consideration.

Sincerely,

Joe


## Convert formats

In [13]:
data_json = {
    "resturant employees": [
        {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
        {"name":"Bob", "email":"bob32@gmail.com"},
        {"name":"Jai", "email":"jai87@gmail.com"}
    ]
}

In [15]:
prompt = f"""
Translate the python dictionary from JSON to an HTML table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<table>
  <caption>Resturant Employees</caption>
  <thead>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>


In [16]:
from IPython.display import display, HTML
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spelling and grammar check

In [17]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]

In [21]:
for t in text:
    prompt = f"""
        Proofread and rewrite the following text in its correct version.
        ```{t}```
    """
    response = get_completion(prompt)
    print(response)
    print("===")

The girl with the black and white puppies has a ball.
===
Yolanda has her notebook.
===
"It's going to be a long day. Does the car need its oil changed?"
===
There goes my freedom. They're going to bring their suitcases.
===
"You're going to need your notebook."
===
"That medicine affects my ability to sleep. Have you heard of the butterfly effect?"
===
This phrase is to check ChatGPT for spelling ability.
===


In [22]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I think there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [24]:
!pip install redlines

In [26]:
from redlines import Redlines
from IPython.display import display, Markdown

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style="color:red;font-weight:700;text-decoration:line-through;">It's </span><span style="color:red;font-weight:700;">Additionally, it's </span>a bit small for what I paid for <span style="color:red;font-weight:700;text-decoration:line-through;">it though. </span><span style="color:red;font-weight:700;">it. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter.
</span><span style="color:red;font-weight:700;">daughter.</span>

In [27]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

Title: A Soft and Cute Panda Plushie for All Ages

As an adult, I can attest that pandas are not just for kids. That's why I got this adorable panda plushie for my daughter's birthday, after she kept taking mine from my room. And let me tell you, it was a hit!

The plushie is super soft and cuddly, making it the perfect companion for my daughter. She takes it everywhere with her, and it has quickly become her favorite toy. However, I did notice that one of the ears is a bit lower than the other, which I don't think was designed to be asymmetrical. But that doesn't take away from its cuteness.

The only downside is that it's a bit small for the price I paid. I think there might be other options that are bigger for the same price. But overall, I'm happy with my purchase.

One thing that surprised me was that it arrived a day earlier than expected. This gave me the chance to play with it myself before giving it to my daughter. And I have to say, I was impressed with the quality and attention to detail.

In conclusion, if you're looking for a soft and cute panda plushie for yourself or a loved one, this is definitely a great option. Just be aware that it might be a bit smaller than expected.